Run cells bellow to preprocess our **custom** datasets.

Estimated time is around 45 minutes.

In [1]:
import pandas as pd
import os
from pathlib import Path

All Items

In [2]:
ds_path = "/home/ubuntu/vasp/Datasets/ml-20m/preprocessed_100"
ds_path_out = "/home/ubuntu/vasp/Datasets/ml-20m/preprocessed_vasp"

# ds_path = "/home/ubuntu/vasp/Datasets/netflix/preprocessed_100"
# ds_path_out = "/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp"

# ds_path = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_100"
# ds_path_out = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp"

Path(ds_path_out).mkdir(parents=True, exist_ok=True)

In [3]:
temp_tr = pd.read_csv(os.path.join(ds_path, 'train_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'train_te.csv'))
train = pd.concat([temp_tr, temp_te])
train.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(train.head())
print(len(train))

temp_tr = pd.read_csv(os.path.join(ds_path, 'validation_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'validation_te.csv'))
val = pd.concat([temp_tr, temp_te])
val.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(val.head())
print(len(val))

temp_tr = pd.read_csv(os.path.join(ds_path, 'test_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'test_te.csv'))
test = pd.concat([temp_tr, temp_te])
test.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(test.head())
print(len(test))

full = pd.concat([train, val, test])
print(full.head())
print(len(full))

   userid  itemid                title
0   65280       0     Grumpier Old Men
1   65280       1   Mr. Holland's Opus
2   65280       2  From Dusk Till Dawn
3   65280       5  Legends of the Fall
4   65280       7         Blade Runner
8002786
   userid  itemid                                              title
0  116149     297  Interview with the Vampire: The Vampire Chroni...
1  116149     621                             Léon: The Professional
2  116149     301                                       Pulp Fiction
3  116149       7                                       Blade Runner
4  116149     205                                           Die Hard
996114
   userid  itemid          title
0  134677     290      GoldenEye
1  134677     178     The Client
2  134677     212   Forrest Gump
3  134677     183          Speed
4  134677     723  Carlito's Way
990897
   userid  itemid                title
0   65280       0     Grumpier Old Men
1   65280       1   Mr. Holland's Opus
2   65280      

In [4]:
items = pd.read_csv(os.path.join(ds_path, 'unique_iId.txt'))
items['iId']=items['iId'].apply(str)
items.rename({'iId': 'itemid'}, axis=1, inplace=True)

items.to_json(os.path.join(ds_path_out, 'items.json'))
items

,itemid
0,3
1,62
2,70
3,110
4,260
...,...
19902,88993
19903,87392
19904,95615
19905,110167


In [5]:
result = train['itemid'].isin(test['itemid'])

result.unique()

array([ True, False])

Purchases and grouped purchases.

In [6]:
interactions = full

interactions['itemid'] = interactions['itemid'].apply(str)
interactions['userid'] = interactions['userid'].apply(str)
interactions.reset_index(drop=True, inplace=True)
interactions.to_json(os.path.join(ds_path_out, 'purchases.json'))

interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userid','itemids']].drop_duplicates()
iii.reset_index(drop=True, inplace=True)
iii.to_json(os.path.join(ds_path_out, 'purchases_txt.json'))
print(iii.head())

  userid                                            itemids
0  65280  0,1,2,5,7,8,9,10,12,13,14,15,16,17,18,19,20,21...
1  74626  46,47,4,48,50,51,6,52,7,8,53,55,56,58,59,60,62...
2  45784  174,1,3,175,176,4,177,48,49,178,179,180,181,18...
3  67928  174,4,211,212,185,6,213,214,193,195,216,201,21...
4  11534  43,0,290,291,292,293,3,294,295,296,5,298,299,3...


Keep only users with 5 or more interactions.

Purchases.

In [7]:
purchases=pd.read_json(os.path.join(ds_path_out, 'purchases.json'))
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
#purchases_item_counts = purchases[['uId','iId']]
#purchases_user_counts = purchases[['uId','iId']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_user_count))
pu5=purchases_user_count#[purchases_user_count.nr_of_purchases>=5]
print(len(purchases_user_count))
print(len(pu5))

purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_item_count_pu5))

#purchases_pu5.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))
purchases.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))

136677
136677
136677
19907


Grouped purchases.

In [8]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userid','itemids']].drop_duplicates()
iii['uId']=iii['userid'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json(os.path.join(ds_path_out, 'purchases_txt_pu5.json'))

Users.

In [9]:
iii['userid'].to_frame().to_json(os.path.join(ds_path_out, 'users_pu5.json'))
print(iii['userid'].to_frame().head())

    userid
0    65280
35   74626
135  45784
175  67928
262  11534


Items

In [10]:
#items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")
items[items.itemid.isin(items.itemid)].to_json(os.path.join(ds_path_out, 'items_pu5.json'))

Items sorted by number of interactions

In [11]:
purchases_item_count_pu5.to_json(os.path.join(ds_path_out, 'items_sorted_pu5.json'))
purchases_item_count_pu5

,itemid,nr_of_purchases
14241,49,55705
12143,301,52265
14686,53,50061
11154,212,47302
13241,4,42578
...,...,...
9126,18210,1
6074,15464,1
6073,15463,1
9129,18213,1


Users sorted by number of interactions

In [12]:
pu5.to_json(os.path.join(ds_path_out, 'users_sorted_pu5.json'))
pu5

,userid,nr_of_purchases
66546,36880,3177
49700,21718,2655
52403,24150,2503
32915,129620,2376
94965,62457,2349
...,...,...
8757,107879,5
102838,69543,5
123544,88179,5
92111,5989,5


Create train, val and test split.

In [13]:
#users = pd.read_json('users_pu5.json')
#shuffled_users = users.sample(frac=1., random_state=42)
test_users = test['userid'].to_frame().drop_duplicates(subset=['userid'])
val_users = val['userid'].to_frame().drop_duplicates(subset=['userid'])
train_users = train['userid'].to_frame().drop_duplicates(subset=['userid'])

test_users.to_json(os.path.join(ds_path_out, "test_users.json"))
print(test_users.head())
val_users.to_json(os.path.join(ds_path_out, "val_users.json"))
print(val_users.head())
train_users.to_json(os.path.join(ds_path_out, "train_users.json"))
print(train_users.head())

len(train_users),len(val_users),len(test_users)

    userid
0   134677
13  123737
25  128041
53  130583
79  128326
     userid
0    116149
71   120983
171  121633
221  115830
229  115556
     userid
0     65280
35    74626
135   45784
175   67928
262   11534


(109342, 13668, 13667)

List of generated json files with preprocessed data.

In [14]:
!ls /home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/*.json

/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/items.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases.json
/home/ubuntu/vasp/Datasets/netflix/preprocessed_vasp/purchases_txt.json
